In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../src/utils")
from utils import read_and_prepare_data, get_array_of_timestemps, SimpleGraphVoltDatasetLoader

In [17]:
#clear gpu memory
import torch
torch.cuda.empty_cache()

In [2]:
from tqdm import tqdm

In [3]:
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
from torch_geometric_temporal.signal import temporal_signal_split

In [4]:
trafo_id = "T1330"

In [5]:
loader = SimpleGraphVoltDatasetLoader(trafo_id)

In [6]:
loader_data = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=4)

(2, 114)
(114, 5)
(114,)
(70065, 113, 21, 12)
(70065, 113, 4)


In [9]:
train_dataset, test_dataset = temporal_signal_split(loader_data, train_ratio=0.8)

In [18]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN

class TemporalGNN(torch.nn.Module):
    def __init__(self, node_features, periods):
        super(TemporalGNN, self).__init__()
        # Attention Temporal Graph Convolutional Cell
        out_channels = 256
        self.tgnn = A3TGCN(in_channels=node_features, 
                           out_channels=out_channels, 
                           periods=periods)
        # Equals single-shot prediction
        self.linear = torch.nn.Linear(out_channels, periods)

    def forward(self, x, edge_index):
        """
        x = Node features for T time steps
        edge_index = Graph edge indices
        """
        h = self.tgnn(x, edge_index)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [ ]:
from torch_geometric.data import InMemoryDataset, Data

In [26]:
# GPU support
device = torch.device('cuda') # cuda
subset = 2000

# Create model and optimizers
model = TemporalGNN(node_features=loader_data[0].x.shape[1], periods=loader_data[0].y.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model.train()
batch_size = 1

epochs_loss = []

print("Running training...")
for epoch in range(2): 
    optimizer.zero_grad()
    epoch_loss = []
    loss = 0
    step = 0
    print(f"--------- Epoch {epoch} ---------")
    for snapshot in tqdm(train_dataset):
        #print(f"--- Step {step} ---")
        snapshot = snapshot.to(device)
        # Get model predictions
        y_hat = model(snapshot.x, snapshot.edge_index)
        # Mean squared error
        intermedaiate_loss = torch.mean((y_hat-snapshot.y)**2) 
        loss = loss + intermedaiate_loss
        step += 1
        # if step%1000==0:
        #   print(f"Intermediate loss at step {step}: {intermedaiate_loss.item()}")
        # # if step > subset:
        #   break
        if step%batch_size == 0:
            loss = loss / batch_size
            loss.backward()
            # print("Epoch {}, batch to {} train MSE: {:.4f}".format(epoch, step, loss.item()))
            epoch_loss.append(loss.detach().cpu().numpy())
            loss = 0
    else: 
        loss = loss / step%batch_size
        loss.backward()
        print("Epoch {}, batch to {} train MSE: {:.4f}".format(epoch, step, loss.item()))
        epoch_loss.append(loss.detach().cpu().numpy())
        loss = 0

    # loss = loss / (step + 1)
    # loss.backward()
    # optimizer.step()
    # optimizer.zero_grad()
    # print("Epoch {} train MSE: {:.4f}".format(epoch, loss.item()))
    # epochs_loss.append(loss.detach().cpu().numpy())
    
    optimizer.step()
    epochs_loss.append(np.mean(epoch_loss))

Running training...
--------- Epoch 0 ---------


0it [00:00, ?it/s]

56052it [05:07, 182.09it/s]


AttributeError: 'float' object has no attribute 'backward'

In [22]:
epochs_loss

[0.9517212,
 0.844185,
 0.7281337,
 0.6892663,
 0.584166,
 0.5243544,
 0.4647213,
 0.4040572,
 0.36148706,
 0.31185463,
 0.2460794,
 0.1925943,
 0.15185113,
 0.12874386,
 0.122715734,
 0.12410098,
 0.13172211,
 0.13668941,
 0.13093674,
 0.11975519]

In [24]:
loader_data[0]

Data(x=[113, 21, 12], edge_index=[2, 114], edge_attr=[114], y=[113, 4])